In [5]:
# Import required libraries
from langchain_ollama import OllamaEmbeddings, OllamaLLM
import chromadb
import os

In [8]:
# Import required libraries
from langchain_ollama import OllamaEmbeddings, OllamaLLM
import chromadb
import os

# Define the LLM model to be used
llm_model = "llama3.2"

# Configure ChromaDB
# Initialize the ChromaDB client with persistent storage in the current directory
chroma_client = chromadb.PersistentClient(path=os.path.join(os.getcwd(), "chroma_db"))

# Define a custom embedding function for ChromaDB using Ollama
class ChromaDBEmbeddingFunction:
    """
    Custom embedding function for ChromaDB using embeddings from Ollama.
    """
    def __init__(self, langchain_embeddings):
        self.langchain_embeddings = langchain_embeddings

    def __call__(self, input):
        # Ensure the input is in a list format for processing
        if isinstance(input, str):
            input = [input]
        return self.langchain_embeddings.embed_documents(input)

# Initialize the embedding function with Ollama embeddings
embedding = ChromaDBEmbeddingFunction(
    OllamaEmbeddings(
        model=llm_model,
        base_url="http://localhost:11434"  # Adjust the base URL as per your Ollama server configuration
    )
)

# Define a collection for the RAG workflow
collection_name = "radiology_rag_collection"
collection = chroma_client.get_or_create_collection(
    name=collection_name,
    metadata={"description": "A collection for RAG with Ollama - Radiology Documents"},
    embedding_function=embedding  # Use the custom embedding function
)

# Function to add documents to the ChromaDB collection
def add_documents_to_collection(documents, ids):
    """
    Add documents to the ChromaDB collection.
    
    Args:
        documents (list of str): The documents to add.
        ids (list of str): Unique IDs for the documents.
    """
    collection.add(
        documents=documents,
        ids=ids
    )

# Example: Radiology-related documents
documents = [
    "Chest X-ray shows a consolidation in the right lower lobe, indicative of pneumonia.",
    "A CT scan of the abdomen reveals a mass in the liver, consistent with a hepatic tumor.",
    "The MRI of the knee demonstrates a tear in the anterior cruciate ligament (ACL).",
    "Chest radiograph shows bilateral pleural effusion and signs of congestive heart failure.",
    "CT angiography indicates a blockage in the left coronary artery, requiring intervention."
]

doc_ids = ["kb1", "kb2", "kb3", "kb4", "kb5"]

# Documents only need to be added once or whenever an update is required. 
# This line of code is included for demonstration purposes:
add_documents_to_collection(documents, doc_ids)

# Function to query the ChromaDB collection
def query_chromadb(query_text, n_results=1):
    """
    Query the ChromaDB collection for relevant documents.
    
    Args:
        query_text (str): The input query.
        n_results (int): The number of top results to return.
    
    Returns:
        list of dict: The top matching documents and their metadata.
    """
    results = collection.query(
        query_texts=[query_text],
        n_results=n_results
    )
    return results["documents"], results["metadatas"]

# Function to interact with the Ollama LLM
def query_ollama(prompt):
    """
    Send a query to Ollama and retrieve the response.
    
    Args:
        prompt (str): The input prompt for Ollama.
    
    Returns:
        str: The response from Ollama.
    """
    llm = OllamaLLM(model=llm_model)
    return llm.invoke(prompt)

# RAG pipeline: Combine ChromaDB and Ollama for Retrieval-Augmented Generation
def rag_pipeline(query_text):
    """
    Perform Retrieval-Augmented Generation (RAG) by combining ChromaDB and Ollama.
    
    Args:
        query_text (str): The input query.
    
    Returns:
        str: The generated response from Ollama augmented with retrieved context.
    """
    # Step 1: Retrieve relevant documents from ChromaDB
    retrieved_docs, metadata = query_chromadb(query_text)
    context = " ".join(retrieved_docs[0]) if retrieved_docs else "No relevant documents found."

    # Step 2: Send the query along with the context to Ollama
    augmented_prompt = f"Context: {context}\n\nQuestion: {query_text}\nAnswer:"
    print("######## Augmented Prompt ########")
    print(augmented_prompt)

    response = query_ollama(augmented_prompt)
    return response

# Example usage
# Define a query to test the RAG pipeline
query = "What does a CT scan of the abdomen reveal?"  # Change the query as needed
response = rag_pipeline(query)
print("######## Response from Radiology Department is ########\n", response)


######## Augmented Prompt ########
Context: The MRI of the knee demonstrates a tear in the anterior cruciate ligament (ACL).

Question: What does a CT scan of the abdomen reveal?
Answer:
######## Response from Radiology Department is ########
 Based on the context, I would expect that a patient with an ACL injury and requiring further imaging would have multiple injuries sustained during the traumatic event, including potential internal injuries to the abdomen.

Therefore, a CT scan of the abdomen may reveal:

* Hemorrhage or blood in the abdominal cavity
* Free fluid in the peritoneal space (indicating bleeding)
* Liver or spleen lacerations
* Fractures or injuries to the ribs or pelvic bones
* Potential injuries to internal organs such as the intestines, kidneys, or liver

Please note that this is a hypothetical scenario, and actual CT scan findings can vary greatly depending on the specific circumstances of the patient's injury.
